# Testing the prediction accuracy

- **02 = calm**
- **03 = happy**
- **04 = sad**
- **05 = angry**
- **06 = fearful**

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
%pylab inline

import librosa
import librosa.display
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

Populating the interactive namespace from numpy and matplotlib


In [2]:
# importing labels of the files
labels = os.listdir("Data/")

In [3]:
# Emotions
emotions = np.array([label.split("-")[2] for label in labels])

In [4]:
def extract_features(filename):
    audio, sample_rate = librosa.load(filename, res_type="kaiser_fast")
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features

In [5]:
features = []

for label in labels:
    filename = f"Data/{label}"
    
    emotion = label.split("-")[2]
    data = extract_features(filename)
    features.append([data, emotion])

In [6]:
features[1]

[array([-6.66113403e+02,  5.62173576e+01, -7.56887484e+00,  1.57707405e+01,
        -8.45027637e+00,  1.83744097e+00, -6.82167864e+00, -4.37701273e+00,
        -5.91381168e+00,  5.17688990e-01, -5.59894037e+00, -2.13708138e+00,
        -6.38166094e+00,  2.84868383e+00, -6.89996719e+00, -1.23033297e+00,
        -4.16035271e+00, -1.37664831e+00, -4.97348595e+00, -1.35401809e+00,
        -2.00252700e+00, -2.14882398e+00, -1.11055171e+00, -2.46265188e-01,
        -1.22439280e-01,  4.77262735e-01,  4.27485657e+00,  4.82439423e+00,
         7.54139376e+00,  4.93660450e+00,  3.66524744e+00,  3.25066614e+00,
         2.14365625e+00,  3.23849964e+00,  2.43824267e+00,  2.52844048e+00,
        -3.09295207e-01,  9.82488215e-01,  1.01983917e+00,  5.99840820e-01],
       dtype=float32),
 '02']

In [7]:
features_df = pd.DataFrame(features, columns=["feature", "emotion"])

In [8]:
X=np.array(features_df['feature'].tolist())
y=np.array(features_df['emotion'].tolist())

In [9]:
X.shape, y.shape

((960, 40), (960,))

In [23]:
emotions

array(['02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '02', '02', '02', '02',
       '02', '02', '02', '02', '02', '02', '02', '0

In [10]:
### Label Encoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(emotions))

In [11]:
model = tf.keras.models.load_model("./saveModel/audio_classification.hdf5")

In [12]:
feature1 = X[0]
feature1 = feature1.reshape(1, 40, 1)

In [13]:
livepreds = model.predict(feature1, batch_size=32, verbose=1)
livepreds

1/1 [==============================] - 1s 1s/step


array([[9.9999976e-01, 8.8638537e-08, 1.5558891e-07, 2.5682223e-08,
        1.8400105e-08]], dtype=float32)

In [14]:
livepreds1=livepreds.argmax(axis=1)
liveabc = livepreds1.astype(int).flatten()
livepredictions = (labelencoder.inverse_transform((liveabc)))
livepredictions

array(['02'], dtype='<U2')

In [15]:
y

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [22]:
emotions_df = pd.DataFrame(emotions, columns=["emotions"])
emotions_df.to_csv("./emotions.csv", index=False)